# 计算图

计算图是用于神经网络BP算法的一种工具，其基本思想是复合函数的链式求导法则，可简化误差反向传播的计算。

## 局部计算节点

计算图将神经网络的推理与学习任务分散各个局部计算节点，通过局部节点的计算结果实现神经网络的推理与学习任务。

- **加法节点**

加法节点的作用是实现以下推理的计算片断，

$$
x + y = z
$$(node-add)

误差的反向传播则将$\frac{\partial L}{\partial z} $乘上以下局部计算结果后，

$$
\begin{split}
\frac{\partial z}{\partial x}&=1\\
\frac{\partial z}{\partial y}&=1\\
\end{split}
$$(node-add-local-comp)

反向传入相应分支，即，$x,y$的各分支分别反向传入$\frac{\partial L}{\partial z}\times 1$。式{eq}`node-add-local-comp`分别对应各个分支的局部梯度计算结果。加法节点的计算图如下所示：

:::{figure-md}
:name: fig-comp-graph-node-plus
![加法节点](../img/node-plus.svg){width=200px}

加法节点
:::

- **乘法节点**

与加法节点类似，实现以下局部计算，

$$
x*y=z
$$(node-mult)

误差反向传播则分别将以下结果反向传入对应分支，即$x$分支传入，

$$
\frac{\partial L}{\partial z}\frac{\partial z}{\partial x}=\frac{\partial L}{\partial z}\cdot y
$$(node-mult-back-x)

$y$分支传入，

$$
\frac{\partial L}{\partial z}\frac{\partial z}{\partial y}=\frac{\partial L}{\partial z}\cdot x
$$(node-mult-back-y)

乘法节点的计算图如下所示：

:::{figure-md} fig-comp-graph-node-times
![乘法节点](../img/node-times.svg){width=200px}

乘法节点
:::

图片的引用{ref}`fig-comp-graph-node-times`

- **分支节点**

分支节点是指相同的值复制后传入各个分支，也称为复制节点。反向传播则是上游传来的梯度之和（与加法节点的运算逻辑正好相反）。

:::{figure-md} fig-comp-graph-node-plus
![分支节点](../img/node-repeat.svg){width=200px}

分支节点示例
:::

当分支扩展到$N$个节点，则可称为**重复节点**。重复节点的反向传播与分支节点类似，是上游所传的梯度之和。



In [1]:
import numpy as np
x = np.random.randn(1, 3)  # 假设的输入数据
y = np.repeat(x, 2, axis=0) # 正向传播

dy=np.random.randn(2, 3) #假设的梯度
dx=np.sum(dy, axis=0, keepdims=True) #梯度传播
print("Input x:\n", x)
print("Output y:\n", y) 
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)

Input x:
 [[ 0.22911169 -1.43466507 -0.83761472]]
Output y:
 [[ 0.22911169 -1.43466507 -0.83761472]
 [ 0.22911169 -1.43466507 -0.83761472]]
Gradient dy:
 [[ 0.46043168  1.54023732  0.17732771]
 [ 0.62153044  1.17685833 -1.77991773]]
Gradient dx:
 [[ 1.08196212  2.71709565 -1.60259002]]


- **sum节点**

sum节点与重复节点正好相反，推理时其输入为各个分支的和，反向传播时各分支传入的值是其上游值的复制。

In [2]:
x = np.random.randn(2, 3)  # 假设的输入数据
y = np.sum(x, axis=0, keepdims=True)  # 正向传播
dy = np.random.randn(1, 3)  # 假设的梯度
dx = np.repeat(dy, 2, axis=0)  # 梯度传播
print("Input x:\n", x)  
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)

Input x:
 [[-0.43673587 -0.17054611 -0.31989842]
 [ 0.70348067  0.88183268 -0.19767539]]
Output y:
 [[ 0.2667448   0.71128656 -0.51757381]]
Gradient dy:
 [[ 0.21866111 -0.21117233 -0.31253189]]
Gradient dx:
 [[ 0.21866111 -0.21117233 -0.31253189]
 [ 0.21866111 -0.21117233 -0.31253189]]


- **MatMul节点**

矩阵乘积节点(假设向量为行向量)，即

$$
\pmb{y}_{1\times H}=\pmb{x}_{1\times D}\pmb{W}_{D\times H}
$$(node-matmul)

该计算结点的难点在于反向传播，即以下偏导数(**分子布局**)的计算，

$$
\begin{split}
\frac{\partial L}{\partial \pmb{x}}_{1\times D}&=\frac{\partial L}{\partial \pmb{y}}\frac{\partial \pmb{y}}{\partial \pmb{x}}   =\left[\frac{\partial L}{\pmb{y}}\right]_{1\times H}\left[\pmb{W}^\top\right]_{H\times D}\\
\left[\frac{\partial L}{\partial \pmb{W}}\right]_{D\times H}&=\frac{\partial L}{\partial \pmb{y}}\frac{\partial\pmb{y}}{\partial \pmb{W}}   =\left[\pmb{x}^\top\right]_{D\times 1} \left[\frac{\partial L}{\partial\pmb{y}}\right]_{1\times H}\\
\end{split}
$$(node-matmul-back)

式{eq}`node-matmul-back`的第1个等式容易实现，略过。第2个等式的推导如下：由矩阵乘法定义可知，

$$
\frac{\partial y_j}{\partial W_{ik}}=\left\{\begin{array}{ll}x_i,&j==k\\ 0,& j\neq k \end{array} \right.
$$

因此，可以得到以下等式，

$$
\frac{\partial L}{\partial W_{ij}}=\sum_k \frac{\partial L}{\partial y_k}\frac{\partial y_k}{\partial W_{ij}}=\frac{\partial L}{\partial y_j}\frac{\partial y_j}{\partial W_{ij}}=\frac{\partial L}{\partial y_j}\cdot x_i
$$

则有梯度如下，

$$
\frac{\partial L}{\partial \pmb{W}}=\left[ \frac{\partial L}{\partial y_j}x_i  \right]_{ij}=\left[\pmb{x}^\top\right]_{D\times 1} \left[\frac{\partial L}{\partial\pmb{y}}\right]_{1\times H}
$$

注意：当$\pmb{x}$是小批量样本时，{eq}`node-matmul-back`形式仍然保持不变。

In [3]:
class MatMul:
    def __init__(self, W):
        self.params = [W]
        self.grads = [np.zeros_like(W)]  #分子布局
        self.x = None
    
    def forward(self, x):
        W, = self.params
        out = np.dot(x, W)  # 矩阵乘法
        self.x = x  #保存输入，反向传播时使用
        return out
    
    def backward(self, dout):
        W, = self.params
        dx = np.dot(dout, W.T)  # 输入梯度
        dW = np.dot(self.x.T, dout)  # 权重梯度
        self.grads[0][...] = dW  # 更新梯度
        return dx
    
# 测试 MatMul 类
W = np.random.randn(3, 4)  # 假设的权重 
x = np.random.randn(2, 3)  # 假设的输入数据
matmul = MatMul(W)  
y = matmul.forward(x)  # 正向传播
dy = np.random.randn(2, 4)  # 假设的梯度    
dx = matmul.backward(dy)  # 反向传播
print("Input x:\n", x)  
print("Weights W:\n", W)
print("Output y:\n", y)     
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)     
print("Gradient dW:\n", matmul.grads[0])  # 权重梯度
    
        

Input x:
 [[0.40429413 0.11323453 0.47863662]
 [0.42221198 1.18253466 1.19820721]]
Weights W:
 [[-0.62936131 -0.77942967  1.36712125  0.29763984]
 [-1.20542526  0.33624955  0.37687015  0.69005388]
 [-0.69869777 -0.7866488   0.10005314 -0.153918  ]]
Output y:
 [[-0.72536519 -0.65356271  0.64328291  0.12480118]
 [-2.52836575 -0.87402607  1.14276138  0.75725409]]
Gradient dy:
 [[ 0.08156153  0.34855414  1.5969379   2.00391902]
 [ 0.64648098  0.40669407  1.40479102 -1.35386023]]
Gradient dx:
 [[ 2.45664878  2.00353516 -0.47983712]
 [ 0.79369737 -1.04734651 -0.42268301]]
Gradient dW:
 [[ 0.30592686  0.3126295   1.23875222  0.23855668]
 [ 0.77372174  0.5203982   1.84204259 -1.37407382]
 [ 0.8136565   0.65413454  2.4475837  -0.66305606]]


## 局部计算的层

通过计算图的计算结点，可以实现一些神经网络的常用层。这些层一般都是结点的组合结果。

- **sigmoid层**

sigmoid层主要由sigmoid函数组成，即

$$
\begin{split}
y&=\frac{1}{\exp(-x)}\\
\frac{\partial y}{\partial x}&=y(1-y)
\end{split}
$$(sigmoid-layer)



In [4]:
class Sigmoid:
    def __init__(self):
        self.params = []    
        self.grads = []
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))  # Sigmoid 函数
        self.out = out  # 保存输出，反向传播时使用
        return out
    
    def backward(self, dout):
        dx = dout * self.out * (1-self.out)  # Sigmoid 的梯度
        return dx

# 测试 Sigmoid 类
x = np.random.randn(2, 3)  # 假设的输入数据
sigmoid = Sigmoid()
y = sigmoid.forward(x)  # 正向传播
dy = np.random.randn(2, 3)  # 假设的梯度
dx = sigmoid.backward(dy)  # 反向传播
print("Input x:\n", x)
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)  # Sigmoid 的梯度


Input x:
 [[-1.17101033  1.66884128 -0.98171431]
 [-1.10026943  0.43641483 -0.04167507]]
Output y:
 [[0.23667241 0.84142127 0.27255176]
 [0.24968942 0.60740443 0.48958274]]
Gradient dy:
 [[-1.18697953  0.97164569  0.61804125]
 [ 0.32952121 -0.06370549 -0.19465746]]
Gradient dx:
 [[-0.21443804  0.12964816  0.12253737]
 [ 0.06173402 -0.01519148 -0.04864324]]


- **仿射层(Affine)**

Affine层主要实现了线性计算的功能，通过矩阵乘法节点和重复节点完成计算功能。

$$
\pmb{z}=\pmb{x}^\top\pmb{W}+\pmb{b}
$$(affine-node)

:::{figure-md} fig-affine
![仿射节点](../img/node-affine.svg){width=600px}

仿射节点
:::

参见图{ref}`fig-affine`的计算过程。该层的计算代码实现如下：

In [5]:
class Affine:
    def __init__(self, W, b):
        self.params = [W, b]
        self.grads = [np.zeros_like(W), np.zeros_like(b)]
        self.x = None   #反向传播时需要
    
    def forward(self, x):
        W, b = self.params
        out = np.dot(x, W) + b
        self.x = x
        return out
    
    def backward(self, dout):
        W, b = self.params
        dx = np.dot(dout, W.T)
        dw = np.dot(self.x.T, dout)
        db = np.sum(dout, axis=0)

        self.grads[0][...]=dw
        self.grads[1][...]=db
        return dx 

#测试Affine节点
W = np.random.randn(3, 4)  # 假设的权重
b = np.random.randn(4)  # 假设的偏置    
x = np.random.randn(2, 3)  # 假设的输入数据
affine = Affine(W, b)
y = affine.forward(x)  # 正向传播
dy = np.random.randn(2, 4)  # 假设的梯度
dx = affine.backward(dy)  # 反向传播
print("Input x:\n", x)
print("Weights W:\n", W)
print("Bias b:\n", b)
print("Output y:\n", y)
print("Gradient dy:\n", dy)
print("Gradient dx:\n", dx)  # 输入梯度
print("Gradient dW:\n", affine.grads[0])  # 权重梯度
print("Gradient db:\n", affine.grads[1])  # 偏置梯度

    

Input x:
 [[-0.68756798  0.43177377 -0.02925057]
 [-1.36102111  0.45077954  1.55719405]]
Weights W:
 [[ 0.14290845 -0.73001669 -1.59705321  0.71603871]
 [ 0.63478605 -0.51794666 -0.41526949 -0.31846297]
 [ 0.09908268 -0.30332353 -0.29725894  1.06398935]]
Bias b:
 [1.43904631 0.13557704 0.30998381 0.18458455]
Output y:
 [[ 1.61197277  0.42274975  1.23745898 -0.47636699]
 [ 1.68498441  0.42333181  1.83352209  0.72332205]]
Gradient dy:
 [[ 0.47736092  1.962861   -0.90886098 -0.24775618]
 [-0.83921138 -0.47721892  1.54217898  0.47660766]]
Gradient dx:
 [[-0.09060606 -0.25731185 -0.54152663]
 [-1.89322497 -1.07774749  0.1102794 ]]
Gradient dW:
 [[ 0.81396632 -0.70009535 -1.47403443 -0.47832386]
 [-0.1721874   0.63239136  0.30276041  0.10787036]
 [-1.32077805 -0.80053728  2.42805664  0.74941762]]
Gradient db:
 [-0.36185046  1.48564208  0.633318    0.22885148]


- **Softmax损失层**

该层主要由softmax函数以及交叉熵损失函数复合而成。softmax函数是指以下函数，

$$
softmax(\pmb{x})=\frac{\exp(\pmb{x})}{\sum_i \exp(x_i)}
$$(softmax-fun-def)

交叉熵损失则是指以下损失函数(单个样本one-hot形式)，

$$
loss(y,t)=\sum_i t_i\log y_i
$$(corss-entropy-def)

In [6]:
def softmax(x):
    if x.ndim == 2:  # 如果是二维数组
        x -= np.max(x, axis=1, keepdims=True)  # 减去每行的最大值
        y = np.exp(x) / np.sum(np.exp(x), axis=1, keepdims=True)  # softmax计算
    else:  # 如果是一维数组
        x -= np.max(x)  # 减去最大值
        y = np.exp(x) / np.sum(np.exp(x))  # softmax计算
    return y

def cross_entropy_loss(y, t):
    if y.ndim == 1:  # 如果是向量
        y = y.reshape(1, -1)  # 转换为二维数组
        t = t.reshape(1, -1)  # 转换为二维数组
    if t.size == y.size:
        t = t.argmax(axis=1)  # 如果t是one-hot编码，转换为类别索引
    batch_size = y.shape[0]  # 获取批大小
    loss = -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size  # 计算交叉熵损失
    return loss

class SoftmaxWithLoss:
    def __init__(self):
        self.params = []
        self.grads = []
        self.y = None  # 保存softmax输出
        self.t = None  # 保存标签
    
    def forward(self, x, t):
        self.t = t  # 保存标签
        self.y = softmax(x)  # 计算softmax输出

        # 在监督标签为one-hot向量的情况下，转换为正确解标签的索引
        if self.t.size == self.y.size:
            self.t = self.t.argmax(axis=1)
        loss = cross_entropy_loss(self.y, self.t)
        return self.y
    
    def backward(self, dout=1):
        batch_size = self.y.shape[0]  # 获取批大小
        dx = self.y.copy()  # 复制softmax输出
        dx[np.arange(batch_size), self.t] -= 1  # 减去正确类别的梯度
        dx *= dout
        dx /= batch_size  # 平均化梯度
        return dx
    
# 测试 SoftmaxWithLoss 类
x = np.random.randn(2, 3)  # 假设的输入数据
t = np.array([0, 1])  # 假设的标签  
softmax_loss = SoftmaxWithLoss()
y = softmax_loss.forward(x, t)  # 正向传播
dy = softmax_loss.backward()  # 反向传播
print("Input x:\n", x)
print("Labels t:\n", t)
print("Softmax Output y:\n", y)
print("Gradient dy:\n", dy)  # softmax的梯度
# 测试 SoftmaxWithLoss 类的交叉熵损失
loss = cross_entropy_loss(y, t)  # 计算交叉熵损失
print("Cross Entropy Loss:\n", loss)  # 输出交叉熵损失
# 测试 SoftmaxWithLoss 类的梯度
print("Gradient dx:\n", dy)  # 输出梯度


Input x:
 [[-2.63092393 -0.02650227  0.        ]
 [-3.61808467 -3.58864891  0.        ]]
Labels t:
 [0 1]
Softmax Output y:
 [[0.03519888 0.47600858 0.48879254]
 [0.02544789 0.0262081  0.94834402]]
Gradient dy:
 [[-0.48240056  0.23800429  0.24439627]
 [ 0.01272394 -0.48689595  0.47417201]]
Cross Entropy Loss:
 3.494210632257319
Gradient dx:
 [[-0.48240056  0.23800429  0.24439627]
 [ 0.01272394 -0.48689595  0.47417201]]


## 优化器


In [ ]:
class SGD:
    '''
    随机梯度下降法（Stochastic Gradient Descent）
    '''
    def __init__(self, lr=0.01):
        self.lr = lr  # 学习率
    
    def update(self, params, grads):
        for i in range(len(params)):
            params[i] -= self.lr * grads[i]  # 更新参数


class Momentum:
    '''
    Momentum SGD
    '''
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
        
    def update(self, params, grads):
        if self.v is None:
            self.v = []
            for param in params:
                self.v.append(np.zeros_like(param))

        for i in range(len(params)):
            self.v[i] = self.momentum * self.v[i] - self.lr * grads[i]
            params[i] += self.v[i]

class AdaGrad:
    '''
    AdaGrad
    '''
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = []
            for param in params:
                self.h.append(np.zeros_like(param))

        for i in range(len(params)):
            self.h[i] += grads[i] * grads[i]
            params[i] -= self.lr * grads[i] / (np.sqrt(self.h[i]) + 1e-7)


class RMSprop:
    '''
    RMSprop
    '''
    def __init__(self, lr=0.01, decay_rate = 0.99):
        self.lr = lr
        self.decay_rate = decay_rate
        self.h = None
        
    def update(self, params, grads):
        if self.h is None:
            self.h = []
            for param in params:
                self.h.append(np.zeros_like(param))

        for i in range(len(params)):
            self.h[i] *= self.decay_rate
            self.h[i] += (1 - self.decay_rate) * grads[i] * grads[i]
            params[i] -= self.lr * grads[i] / (np.sqrt(self.h[i]) + 1e-7)


class Adam:
    '''
    Adam (http://arxiv.org/abs/1412.6980v8)
    '''
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
        
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = [], []
            for param in params:
                self.m.append(np.zeros_like(param))
                self.v.append(np.zeros_like(param))
        
        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)

        for i in range(len(params)):
            self.m[i] += (1 - self.beta1) * (grads[i] - self.m[i])
            self.v[i] += (1 - self.beta2) * (grads[i]**2 - self.v[i])
            
            params[i] -= lr_t * self.m[i] / (np.sqrt(self.v[i]) + 1e-7)


## 训练


In [ ]:
from matplotlib import pyplot as plt
import time

def remove_duplicates(params, grads):
    params, grads = params[:], grads[:]  # copy list

    while True:
        find_flg = False
        L = len(params)

        for i in range(0, L - 1):
            for j in range(i + 1, L):
                # 在共享权重的情况下
                if params[i] is params[j]:
                    grads[i] += grads[j]  # 加上梯度
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)
                # 在作为转置矩阵共享权重的情况下（weight tying）
                elif params[i].ndim == 2 and params[j].ndim == 2 and \
                     params[i].T.shape == params[j].shape and np.all(params[i].T == params[j]):
                    grads[i] += grads[j].T
                    find_flg = True
                    params.pop(j)
                    grads.pop(j)
                if find_flg: break
            if find_flg: break
        if not find_flg: break
    return params, grads

def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / (total_norm + 1e-6)
    if rate < 1:
        for grad in grads:
            grad *= rate


class Trainer:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        self.loss_list = []  # 存储损失值
        self.eval_interval = None
        self.current_epoch = 0
    
    def fit(self, x, t, max_epoch=10, batch_size=32, max_grad=None, eval_interval=20):
        data_size = len(x)
        max_iters = data_size // batch_size
        self.eval_interval= eval_interval
        model, optimizer = self.model, self.optimizer
        total_loss = 0.0
        loss_count = 0
        start_time = time.time()
        for epoch in range(max_epoch):
            idx = np.random.permutation(data_size)
            x = x[idx]
            t = t[idx]
            for iters in range(max_iters):
                batch_x = x[iters * batch_size:(iters + 1) * batch_size]
                batch_t = t[iters * batch_size:(iters + 1) * batch_size]

                loss = model.forward(batch_x, batch_t)  # 正向传播
                model.backward()
                params, grads = remove_duplicates(model.params, model.grads)  # 获取参数和梯度
                if max_grad is not None:
                    clip_grads(grads, max_grad)
                optimizer.update(params, grads)
                total_loss += loss
                loss_count += 1
                # 评价
                if (eval_interval is not None) and (iters % eval_interval) == 0:
                    avg_loss = total_loss / loss_count
                    elapsed_time = time.time() - start_time
                    print('| epoch %d |  iter %d / %d | time %d[s] | loss %.2f'
                          % (self.current_epoch + 1, iters + 1, max_iters, elapsed_time, avg_loss))
                    self.loss_list.append(float(avg_loss))
                    total_loss, loss_count = 0, 0

            self.current_epoch += 1

    def plot(self, ylim=None):
        x = np.arange(len(self.loss_list))
        if ylim is not None:
            plt.ylim(*ylim)
        plt.plot(x, self.loss_list, label='train')
        plt.xlabel('iterations (x' + str(self.eval_interval) + ')')
        plt.ylabel('loss')
        plt.show()
